# 1 percent CO2 run for carbon cycle metrics

Could add this to the examples

In [ ]:
import copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import time
import scipy.stats
from tqdm import tqdm

from fair21 import (
    SpeciesID, Category, Config, Species, RunMode, Scenario, ClimateResponse, RunConfig, CH4LifetimeMethod, FAIR
)
from fair21.defaults import species_config_from_default

In [ ]:
species_ids = {
    # Greenhouse gases
    'CO2': SpeciesID('CO2', Category.CO2, run_mode=RunMode.CONCENTRATION),
    'CH4': SpeciesID('CH4', Category.CH4, run_mode=RunMode.CONCENTRATION),
    'N2O': SpeciesID('N2O', Category.N2O, run_mode=RunMode.CONCENTRATION),
}

In [ ]:
scenarios_to_include = ['1pctCO2']

In [ ]:
concentration_species = ['CO2', 'CH4', 'N2O']
species_to_include = concentration_species

In [ ]:
df_conc = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')
df_conc.loc[(df_conc['Scenario']=='1pctCO2') & (df_conc['Variable'].str.endswith("|CO2")),'1850']

In [ ]:
scenarios = []
for iscen, scenario in enumerate(scenarios_to_include):
    list_of_species = []
    for ispec, species in enumerate(concentration_species):
        species_rcmip_name = species.replace("-", "")
        if species == 'NOx_aviation':
            species_rcmip_name = 'NOx|MAGICC Fossil and Industrial|Aircraft'
        elif species == 'CO2_FFI':
            species_rcmip_name = 'CO2|MAGICC Fossil and Industrial'
        elif species == 'CO2_AFOLU':
            species_rcmip_name = 'CO2|MAGICC AFOLU'
        conc_in = df_conc.loc[
            (df_conc['Scenario']==scenario) & (df_conc['Variable'].str.endswith("|"+species_rcmip_name)) & 
            (df_conc['Region']=='World'), '1850':'2000'
        ].interpolate(axis=1).values.squeeze()

        list_of_species.append(Species(species_ids[species], concentration=conc_in))
        #print(species_rcmip_name, emis_in.shape)
        
    scenarios.append(Scenario(scenario, list_of_species))